In [1]:
import cv2
import numpy as np
import sys
import os
from tqdm import tqdm

In [2]:
dir = "Thumbnail"
# If thumbnail folder doesn't exist, create it
if not os.path.exists(dir):
    os.makedirs(dir)

In [85]:
# filename = str(sys.argv[1])
filename = "Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】-oEkGC2HV7rc.mp4"
resolution = 240
framerate = 1
x = 5
y = 5

In [91]:
filen = filename[0:filename.rindex('.')]
vidcap = cv2.VideoCapture(filename)

In [92]:
totalFrame = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = vidcap.get(cv2.CAP_PROP_FPS)
print(totalFrame)
print(fps)
print(totalFrame / fps)
print(totalFrame / fps / 5)
vidcap.set(cv2.CAP_PROP_POS_MSEC,totalFrame / fps * 1000)
print("{:.3f}".format(totalFrame/fps))

7044.0
29.97002997002997
235.03480000000002
47.00696000000001
235.035


In [93]:
def captureFrame(vidcap, mil):
  vidcap.set(cv2.CAP_PROP_POS_MSEC, mil)
  _, image = vidcap.read()
  return image

In [111]:
def printTime(src,dst,img,x,y,width,height,count,f):
    print("\n{:02d}:{:02d}:{:02d}.{:03d} --> {:02d}:{:02d}:{:02d}.{:03d}".format(
        int(src/1000/60/60),
        int((src/1000/60)%60),
        int((src/1000)%60),
        int(src%1000),
        int(dst/1000/60/60),
        int((dst/1000/60)%60),
        int((dst/1000)%60),
        int(dst%1000)
    ))
    print("{}#xywh={},{},{},{}".format(img,count%x*width,int(count/x)*height,width,height))

    # f.write("\n\n{:02d}:{:02d}:{:02d}.{:03d} --> {:02d}:{:02d}:{:02d}.{:03d}".format(
    #     int(src/1000/60/60),
    #     int((src/1000/60)%60),
    #     int((src/1000)%60),
    #     int(src%1000),
    #     int(dst/1000/60/60),
    #     int((dst/1000/60)%60),
    #     int((dst/1000)%60),
    #     int(dst%1000)
    # ))
    # f.write("\n{}#xywh={},{},{},{}".format(img,count%x*width,int(count/x)*height,width,height))

In [113]:
def createThumbnails(vidcap, resolution = 240, framerate = 1, x = 5, y = 5):

  scale = resolution / vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)
  width = int(scale * vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(scale * vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  dim = (width,height)

  count = 0
  count_im = 0
  sec = 0
  mil = 0 

  im_h = []
  im_v = []

  f = open("{}/{}.vtt".format(dir,filen), "w")
  f.write("WEBVTT")

  trigger = False

  for i in tqdm(range(int(totalFrame / fps / framerate))):

    mil = sec * 1000

    # Get image and resize
    image = captureFrame(vidcap, mil)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    printTime(
      mil,
      mil+framerate*1000,
      "{}_{}.jpg".format(filen, count_im+1),
      x, y, width, height, count, f)

    # Add image to array
    im_h.append(image)

    # Add time
    count = count + 1
    sec = sec + framerate

    # If 1 row finished
    if (count != 0 and count % x == 0):
      image = cv2.hconcat(im_h)
      im_v.append(image)
      im_h.clear()

      # If 1 image finished
      if (count / x == y):
        count = 0
        count_im = count_im + 1
        cv2.imwrite("{}/{}_{}.jpg".format(dir,filen, count_im),cv2.vconcat(im_v))
        im_v.clear()

  # If not round
  if (sec < totalFrame / fps):
    mil = sec * 1000

    # Get image and resize
    image = captureFrame(vidcap, mil)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    printTime(
      mil,
      totalFrame/fps*1000,
      "{}_{}.jpg".format(filen, count_im+1),
      x, y, width, height, count, f)

    # Add image to array
    im_h.append(image)

    # Add time
    count = count + 1
    sec = sec + framerate

    # If 1 row finished
    if (count != 0 and count % x == 0):
      image = cv2.hconcat(im_h)
      im_v.append(image)
      im_h.clear()

      # If 1 image finished
      if (count / x == y):
        count = 0
        count_im = count_im + 1
        cv2.imwrite("{}/{}_{}.jpg".format(dir, filen, count_im),cv2.vconcat(im_v))
        im_v.clear()
  
  while (count % x != 0):
    trigger = True
    image = np.zeros([height, width, 3], np.uint8)
    im_h.append(image)
    count = count + 1

  if (trigger):
    image = cv2.hconcat(im_h)
    im_v.append(image)
    count_im = count_im + 1
    cv2.imwrite("{}/{}_{}.jpg".format(dir, filen, count_im),cv2.vconcat(im_v))
  elif (count == x):
    count_im = count_im + 1
    cv2.imwrite("{}/{}_{}.jpg".format(dir, filen, count_im),cv2.vconcat(im_v))

  f.close()

createThumbnails(vidcap, x = 10, y = 10, framerate=1)

78,1440,426,240

00:01:04.000 --> 00:01:05.000
Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】-oEkGC2HV7rc_1.jpg#xywh=1704,1440,426,240

00:01:05.000 --> 00:01:06.000
Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】-oEkGC2HV7rc_1.jpg#xywh=2130,1440,426,240

00:01:06.000 --> 00:01:07.000
Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】-oEkGC2HV7rc_1.jpg#xywh=2556,1440,426,240

00:01:07.000 --> 00:01:08.000
Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】-oEkGC2HV7rc_1.jpg#xywh=2982,1440,426,240

00:01:08.000 --> 00:01:09.000
Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】-oEkGC2HV7rc_1.jpg#xywh=3408,1440,426,240

00:01:09.000 --> 00:01:10.000
Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】-oEkGC2HV7rc_1.jpg#xywh=3834,1440,426,240

00:01:10.000 --> 00:01:11.000
Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】-oEkGC2HV7rc_1.jpg#xywh=0,1680,426,240

00:01:11.000 --> 00:01:12.000
Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】-oEkGC2HV7rc_1.jpg#xywh=426,1680,426,240

00:01:12.000 --> 00:01:13.000
Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】-oEk

In [110]:
# filename
# filen